In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
import re
import time

profile = webdriver.ChromeOptions()
profile.add_argument('start-maximized')
profile.add_argument('headless')
profile.add_experimental_option("prefs", {"profile.managed_default_content_settings.images": 2})
profile.add_experimental_option('excludeSwitches', ['enable-logging'])
profile.page_load_strategy = 'eager'

driver = webdriver.Chrome(options=profile)

In [ ]:
info = open(f'link_info.csv', 'w', encoding='utf-8')
info.write("links")

for i in range(1, 41):
    time.sleep(2)
    if i % 10 == 0:
        driver.quit()
        driver = webdriver.Chrome(options=profile)
    driver.get(f'https://www.litres.ru/genre/programmirovanie-5272/?page={i}')
    links = driver.find_elements(by=By.XPATH, value="//div[@class='ArtInfo_wrapper__GoMsb']/a")
    for lin in links:
        info.write(lin.get_attribute("href") + "\n")
info.close()

In [1]:
import requests
import json
import pandas as pd
import numpy as np

data = pd.read_csv("link_info.csv")
book_codes = [d.strip("/").split("-")[-1] for d in data['links'].tolist()]

finalized_reviews = []

prices = []
names = []
authors = []
ratings = []
ratings_cnts = []
review_cnts = []
years = []
ages = []
pages_num = []

for code in book_codes:
    time.sleep(2)
    page = requests.get(f"https://api.litres.ru/foundation//api/arts/{code}/reviews?o=popular&limit=1000")
    response_data = json.loads(page.text)
    response_data = response_data['payload']['data']
    review_cnts.append(len(response_data))
    finalized_reviews.append([data['text'].replace("<p>", "").replace("</p>", "").replace("<div>", "").replace("</div>", "").replace("<br />", ". ").replace("<br/>", ". ").strip() for data in response_data])

    page = requests.get(f"https://api.litres.ru/foundation//api/arts/{code}")
    response_data = json.loads(page.text)
    response_data = response_data['payload']['data']

    prices.append(response_data["prices"]["full_price"])
    names.append(response_data['title'])
    authors.append(response_data['persons'][0]["full_name"])
    ratings.append(response_data['rating']['rated_avg'])
    ratings_cnts.append(response_data['rating']['rated_total_count'])
    ages.append(response_data['min_age'])
    if not response_data["date_written_at"]:
        years.append(np.nan)
    else:
        years.append(response_data["date_written_at"].split("-")[0])
    pages_num.append(response_data['symbols_count'])

data['reviews'] = finalized_reviews
data['name'] = names
data['author'] = authors
data['price'] = prices
data['rating'] = ratings
data['rating_cnt'] = ratings_cnts
data['review_cnt'] = review_cnts
data['year'] = years
data['age'] = ages
data['page'] = pages_num
data.to_csv('final_df.csv', index=False)